##### import DataConnection as Dc
from collections import defaultdict
import time
import datetime
import operator
import csv



pgo = Dc.PGConnection('CRASHDATA', 'shostetter')  #Database Connection
dbo = Dc.DBConnection('dotgissql01', 'gisgrid', 'GISUSER', 'GISUSER') #Database Connection
dbo2 = Dc.DBConnection('DOTDEV55SQL02', 'DataWarehouse', 'SHostetter', 'shostetter') #Database Connection



def get_intersection_universe(pg):
    cur = pg.conn.cursor()
    #.conn.cursor() Handles the connection to a PostgreSQL database instance. It encapsulates a database session. Return a new cursor object using the connection.

    cur.execute("select nodeid, masterid, is_int  from node where is_int = true")#Execute a database operation (query or command)
    # dictionary of nodeid: [masterid, intersection]
    nodes = defaultdict(list, ((int(row[0]), [row[1], row[2]]) for row in cur.fetchall()))
    # cur.fetchall() Fetch all (remaining) rows of a query result, returning them as a list of tuples. An empty list is returned if there is no more record to fetch.
    del cur
    #deletes cur to reset the memory for a new query
    return nodes

In [3]:
node_dict = get_intersection_universe(pgo)
node_dict

defaultdict(list,
            {9043971: [9043971, True],
             14: [14, True],
             18: [18, True],
             19: [19, True],
             22: [22, True],
             23: [23, True],
             25: [25, True],
             26: [26, True],
             27: [27, True],
             9043999: [9044000, True],
             9044000: [9044000, True],
             9044003: [9044003, True],
             9044004: [9044004, True],
             47: [47, True],
             51: [51, True],
             52: [52, True],
             53: [53, True],
             54: [54, True],
             55: [55, True],
             56: [56, True],
             57: [57, True],
             59: [59, True],
             62: [62, True],
             63: [63, True],
             64: [64, True],
             65: [65, True],
             66: [66, True],
             67: [67, True],
             68: [68, True],
             69: [69, True],
             70: [70, True],
             71: [71, True],
    

In [4]:
def alt_get_signalized_int(db, pg, nodes):
    cur = db.conn.cursor()
    cur.execute("""SELECT [PSGM_ID],[ContrType],[NodeID],[Longitude],[Latitude]
                    FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                    --[LION_CURRENT].[GISADMIN].[SIG_TRpsgm] 
                    where NormalizedType != 'Z'
                    --and [Longitude] not in ('','S' ) and  [Latitude] not in ('')
                    and [Longitude] is not null
                    and psgm_id in (41084, 41083)""")
    for row in cur .fetchall():
        psgm_id, contrtype, nodeid, x, y = row
        #print('****',psgm_id, contrtype, nodeid, x, y,'*********')
        nodeid = alt_get_node_from_signal_coords(pg, str(x), str(y))
        #print('-----------',nodeid,'--------------')
              
        if nodeid:
            nodes[int(nodeid)].append('Signal')
    del cur
    return nodes


def alt_get_node_from_signal_coords(pg, x, y):
    cur = pg.conn.cursor()
    cur.execute("""select nodeid, st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) as distance
                    from node where st_distance(geom, st_transform(st_geomfromtext(
                        'SRID=4326;POINT(%s %s)'), 2263)) < 1000
                    and is_int = true
                    order by geom <#> st_transform(st_geomfromtext('SRID=4326;POINT(%s %s)'), 2263) limit 1
                """ % (x, y, x, y, x, y)
                )
    row = cur.fetchone()
    del cur
    if row:
        return int(row[0])
    else:
        return None
# make list of unsignalized masterids

In [5]:
node_dict = alt_get_signalized_int(dbo, pgo, node_dict)
node_dict

defaultdict(list,
            {9043971: [9043971, True],
             14: [14, True],
             18: [18, True],
             19: [19, True],
             22: [22, True],
             23: [23, True],
             25: [25, True],
             26: [26, True],
             27: [27, True],
             9043999: [9044000, True],
             9044000: [9044000, True],
             9044003: [9044003, True],
             9044004: [9044004, True],
             47: [47, True],
             51: [51, True],
             52: [52, True],
             53: [53, True],
             54: [54, True],
             55: [55, True],
             56: [56, True],
             57: [57, True],
             59: [59, True],
             62: [62, True],
             63: [63, True],
             64: [64, True],
             65: [65, True],
             66: [66, True],
             67: [67, True],
             68: [68, True],
             69: [69, True],
             70: [70, True],
             71: [71, True],
    

In [6]:
def get_unsignalized_masterids(nodes):
    mids = defaultdict(list)
    to_remove = set()
    for node in nodes.keys():
        mids[nodes[node][0]].append(node)
        if nodes[node][-1] == 'Signal':
            to_remove.add(nodes[node][0])
    for m in to_remove:
        del(mids[m])
    return mids

In [7]:
masters = get_unsignalized_masterids(node_dict)
masters

defaultdict(list,
            {9043971: [9043971, 44038],
             14: [14],
             18: [18],
             19: [19],
             22: [22],
             23: [23],
             25: [25],
             26: [26],
             27: [27],
             9044000: [9043999, 9044000, 53354, 101411, 101412],
             9044003: [9044003, 3830],
             9044004: [9044004],
             47: [47],
             51: [51],
             52: [52],
             53: [53],
             54: [54],
             55: [55],
             56: [56],
             57: [57],
             59: [59],
             62: [62],
             63: [63],
             64: [64],
             65: [65],
             66: [66],
             67: [67],
             68: [68],
             69: [69],
             70: [70],
             71: [71],
             72: [72],
             74: [74],
             75: [75],
             76: [76],
             77: [77],
             78: [78],
             79: [79],
             80: [80],


In [8]:
def get_all_crashes_nypd(db2):
    # test speed of doing 1 query and iterating in python memory
    cur = db2.conn.cursor()
    cur.execute("""SELECT distinct c.[ANUM_PCT]+c.[ANUM_YY]+c.[ANUM_SEQ] as crashid ,
                    c.[NODEID], cast(c.[OCCURRENCE_DATETIME] as DATE) as accd_date,
                    c.[INJURED_COUNT],c.[KILLED_COUNT],
                    case when vic.ped_nonped = 'P' then 1
                    when vic.PED_NONPED = 'B' then 2 else 3 end as inj_mode,
                    c.[ACCIDENT_DIAGRAM]  AS [COLLISION_TYP],
                    null as ext_of_inj, 0 as pre_action, 0 as direction, 0 as contrib
                    FROM [DataWarehouse].[dbo].[AIS_PD_Core_F] AS c
                    LEFT OUTER JOIN (
                        SELECT [ANUM_PCT], [ANUM_YY], [ANUM_SEQ],
                        MAX(CASE WHEN PRE_ACDNT_ACTION = '10' THEN 1
                        WHEN PRE_ACDNT_ACTION in ('15', '08', '09', '14', '04') THEN 1 -- removes other non-preventable
                        ELSE 0 END) AS PK
                        FROM [DataWarehouse].dbo.AIS_PD_Vehicle_F
                        GROUP BY [ANUM_PCT], [ANUM_YY], [ANUM_SEQ]
                    )AS v -- parked vehicles
                    ON c.[ANUM_PCT] = v.ANUM_PCT AND c.[ANUM_YY] = v.ANUM_YY AND c.[ANUM_SEQ] = v.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Pedestrian_F AS P
                    ON c.[ANUM_PCT] = P.ANUM_PCT AND c.[ANUM_YY] = P.ANUM_YY AND c.[ANUM_SEQ] = P.ANUM_SEQ
                    LEFT OUTER JOIN DataWarehouse.dbo.AIS_PD_Victim_F as vic
                    ON c.ANUM_PCT = vic.ANUM_PCT and c.ANUM_SEQ = vic.ANUM_SEQ and c.ANUM_YY = vic.ANUM_YY
                    WHERE C.OCCURRENCE_DATETIME > (select dateadd(week, -3, dateadd(year, -3, getdate())))
                    AND C.NODEID != 0
                    and v.PK != 1
                """)
    data = cur.fetchall()
    del cur
    # clean up dates
    for row in data:
        d = row[2]  # date field
        row[2] = datetime.datetime.strptime(d, "%Y-%m-%d").date()
    assert isinstance(data, object)
    return data

In [9]:
crash_list = get_all_crashes_nypd(dbo2)  # NYPD data
crash_list

[(u'0011610520', 20505, datetime.date(2016, 5, 2), 0, 0, 3, u'5', None, 0, 0, 0),
 (u'0011610532', 20597, datetime.date(2016, 5, 4), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610534', 19992, datetime.date(2016, 5, 4), 0, 0, 3, u'1', None, 0, 0, 0),
 (u'0011610558', 20371, datetime.date(2016, 5, 6), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610571', 20365, datetime.date(2016, 5, 9), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610572', 15354, datetime.date(2016, 5, 9), 0, 0, 3, u'3', None, 0, 0, 0),
 (u'0011610585', 9030777, datetime.date(2016, 5, 11), 0, 0, 3, u'9', None, 0, 0, 0),
 (u'0011610588', 20247, datetime.date(2016, 5, 11), 0, 0, 3, u'3', None, 0, 0, 0),
 (u'0011610589', 95581, datetime.date(2016, 5, 11), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610594', 20381, datetime.date(2016, 5, 11), 0, 0, 3, u'2', None, 0, 0, 0),
 (u'0011610597', 15357, datetime.date(2016, 5, 12), 0, 0, 3, u'9', None, 0, 0, 0),
 (u'0011610598', 15268, datetime.date(2016, 5, 12), 0, 0, 3, u'9', None, 0, 0, 0),
 (u'0011

In [10]:
def get_all_dir_right_angle_crashes_nypd(db2):
    cur = db2.conn.cursor()
    cur.execute("""/* conservative */
                    -- north / south
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                        and PRE_ACDNT_ACTION != '10'
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3, 7)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- east / west
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (3,7)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (1, 5)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northeast / southwest
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER

                    union
                    -- northwest / southeast
                    select distinct n.ANUM_PCT+ n.[ANUM_YY]+ n.[ANUM_SEQ] as crashid
                    --n.*, o.DIRECTION_OF_TRAVEL
                    from (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (4, 8)
                    ) as n
                    join (
                        SELECT ANUM_PCT, ANUM_YY, ANUM_SEQ, VEHICLE_NUMBER, DIRECTION_OF_TRAVEL
                        FROM [DataWarehouse].[dbo].[AIS_PD_Vehicle_F]
                        where [DIRECTION_OF_TRAVEL] is not null and [DIRECTION_OF_TRAVEL] != ' '
                        and [DIRECTION_OF_TRAVEL] in (2, 6)
                    ) as o
                    on n.ANUM_PCT = o.ANUM_PCT and n.ANUM_SEQ = o.ANUM_SEQ and n.ANUM_YY = o.ANUM_YY
                        and n.VEHICLE_NUMBER != o.VEHICLE_NUMBER""")
    ra = cur.fetchall()
    del cur
    return set([i[0] for i in ra])

In [11]:
drac = get_all_dir_right_angle_crashes_nypd(dbo2)  # NYPD data
drac

{u'0431702557',
 u'0431702556',
 u'1231610263',
 u'0521703263',
 u'1221700085',
 u'0471703062',
 u'0731611031',
 u'0131701166',
 u'1211501863',
 u'1231800286',
 u'0661502918',
 u'0471703069',
 u'0431702558',
 u'1051705931',
 u'0061700629',
 u'0401501429',
 u'1151702803',
 u'1111702063',
 u'1041801232',
 u'1201600009',
 u'1081802371',
 u'1151701712',
 u'1141700915',
 u'0061800492',
 u'0061800493',
 u'1141703999',
 u'0191610964',
 u'0421701104',
 u'0661502913',
 u'0491610119',
 u'0181503926',
 u'0191610963',
 u'1051801717',
 u'1011600056',
 u'1081701185',
 u'0491800436',
 u'1141701848',
 u'1211800202',
 u'1041613421',
 u'1061600243',
 u'1061800241',
 u'1061600241',
 u'1201702230',
 u'0071610820',
 u'1211801156',
 u'1111502957',
 u'1211801152',
 u'0631610685',
 u'0061800728',
 u'0141503683',
 u'1151612761',
 u'0241501169',
 u'0431501973',
 u'0141503684',
 u'0191700191',
 u'0661800274',
 u'0661702131',
 u'0661702132',
 u'0941611311',
 u'0171801610',
 u'0671610830',
 u'1041701595',
 u'12016

In [12]:
class Intersection:
    def __init__(self, masterid, nodeid_list, crashes_list):
        self.masterid = masterid
        self.nodeid_list = nodeid_list
        self.crashes = crashes_list
        self.one_year_crashes = dict()
        self.twelve_month_period()
        self.warrant()
        self.get_12_month_crashes()

    def five_or_more(self):
        count = 0
        for c in self.crashes:
            if c.preventable():
                count += 1
        if count > 4:
            return True
        else:
            return False

    def twelve_month_period(self):
        cl = [[c.crashid, c.ac_date] for c in self.crashes if c.preventable()]
        cl = sorted(cl, key=operator.itemgetter(1), reverse=True)
        while len(cl) > 1:
            if (cl[0][1] - cl[-1][1]).days / 365.0 > 1:
                cl.pop()
            else:
                break
        return cl

    def warrant(self):
        if len(self.twelve_month_period()) > 4:
            return True
        else:
            return False

    def get_12_month_crashes(self):
        crashes = dict((c[0], None) for c in self.twelve_month_period())
        for crash in self.crashes:
            if crash.crashid in crashes.keys():
                crashes[crash.crashid] = crash
        return crashes


In [13]:
def add_intersection_objects_to_master_dict(mids):
    for m in mids:
        mids[m].append(Intersection(m, mids[m], list()))
        if isinstance(mids[m][-1].nodeid_list[-1], Intersection):
            mids[m][-1].nodeid_list = mids[m][-1].nodeid_list[:-1]
            


In [14]:
add_intersection_objects_to_master_dict(masters)

In [15]:
class Crash:
    def __init__(self, crashid, nodeid, ac_date, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac):
        """
        :param crashid:
        :param nodeid:
        :param ac_date:
        :param inj:
        :param fat:
        :param mode:
        :param col_typ:
        :param sev:
        :param pre_actions:
        :param directions:
        :param contrib_fac:
        :return:
        """
        self.crashid = crashid
        self.nodeid = nodeid
        self.inj = inj
        self.fat = fat
        self.mode = mode
        self.col_typ = str(col_typ)
        self.pre_actions = pre_actions
        self.directions = directions
        self.ac_date = ac_date
        self.sev = sev
        self.contib_fac = contrib_fac
        self.right_angle()
        self.ped_bike()
        self.preventable()

    def right_angle(self):
        """
        includes: right angle (4), left turn with (0), and right turn with (5)
        :return: Boolean
        """
        if self.col_typ in ('4', '0', '5'):  # NYPD
            return True
        elif self.col_typ in ('04', '10', '05'):  # NYSDOT
            return True
        else:
            return False

    def ped_bike(self):
        if self.mode in (1, 2):
            return True
        else:
            return False

    def directions_prevent(self):
        if self.directions:
            return True
        else:
            return False

    def preventable(self):
        if self.right_angle() or self.ped_bike() or self.directions_prevent():
            if self.directions_prevent() and not self.right_angle():
                print '%s RA (%s), PB (%s), DIR(%s)' % (str(self.crashid),
                                                    str(self.right_angle()),
                                                    str(self.ped_bike()),
                                                    str(self.directions_prevent())
                                                    )
            return True
        else:
            return False

In [16]:
def add_crashes_to_intersections(nodes, mids, crashes, drac):
    for crash in crashes:
        crashid, nodeid, ac_date, inj, fat, mode, col_typ, sev, pre_actions, directions, contrib_fac = crash
        if crashid in drac:
            directions = True
        else:
            directions = False
        if nodes[int(nodeid)]:
            master = nodes[int(nodeid)][0]
            if master in mids.keys():  # check for unsignalized
                mids[master][-1].crashes.append(Crash(crashid, nodeid, ac_date, inj, fat, mode,
                                                      col_typ, sev, pre_actions, directions, contrib_fac))

In [17]:
add_crashes_to_intersections(node_dict, masters, crash_list, drac)


0011610559 RA (False), PB (False), DIR(True)
0011610605 RA (False), PB (False), DIR(True)
0011610759 RA (False), PB (True), DIR(True)
0011611198 RA (False), PB (False), DIR(True)
0011611206 RA (False), PB (True), DIR(True)
0011611259 RA (False), PB (False), DIR(True)
0011611000 RA (False), PB (False), DIR(True)
0011611076 RA (False), PB (False), DIR(True)
0011611094 RA (False), PB (False), DIR(True)
0011611070 RA (False), PB (False), DIR(True)
0011611093 RA (False), PB (True), DIR(True)
0011611175 RA (False), PB (False), DIR(True)
0011611195 RA (False), PB (False), DIR(True)
0011611212 RA (False), PB (False), DIR(True)
0011611331 RA (False), PB (True), DIR(True)
0011611546 RA (False), PB (False), DIR(True)
0011611369 RA (False), PB (True), DIR(True)
0011611404 RA (False), PB (True), DIR(True)
0011611627 RA (False), PB (False), DIR(True)
0011611792 RA (False), PB (False), DIR(True)
0011611750 RA (False), PB (False), DIR(True)
0011611781 RA (False), PB (False), DIR(True)
0011611809 RA (F

0051700006 RA (False), PB (False), DIR(True)
0051700333 RA (False), PB (False), DIR(True)
0051700582 RA (False), PB (False), DIR(True)
0051700615 RA (False), PB (False), DIR(True)
0051700653 RA (False), PB (False), DIR(True)
0051700702 RA (False), PB (False), DIR(True)
0051700140 RA (False), PB (False), DIR(True)
0051700833 RA (False), PB (False), DIR(True)
0051701061 RA (False), PB (False), DIR(True)
0051701228 RA (False), PB (False), DIR(True)
0061700332 RA (False), PB (False), DIR(True)
0061700666 RA (False), PB (False), DIR(True)
0061700823 RA (False), PB (False), DIR(True)
0061701197 RA (False), PB (False), DIR(True)
0051700999 RA (False), PB (False), DIR(True)
0051701233 RA (False), PB (False), DIR(True)
0051701368 RA (False), PB (True), DIR(True)
0051701434 RA (False), PB (True), DIR(True)
0051701629 RA (False), PB (False), DIR(True)
0051701744 RA (False), PB (False), DIR(True)
0051701767 RA (False), PB (False), DIR(True)
0051701841 RA (False), PB (False), DIR(True)
0051702032 R

0091800746 RA (False), PB (True), DIR(True)
0091800855 RA (False), PB (False), DIR(True)
0101610998 RA (False), PB (False), DIR(True)
0071800262 RA (False), PB (False), DIR(True)
0071800529 RA (False), PB (False), DIR(True)
0071800540 RA (False), PB (False), DIR(True)
0101610681 RA (False), PB (False), DIR(True)
0101611457 RA (False), PB (False), DIR(True)
0101611481 RA (False), PB (True), DIR(True)
0101611640 RA (False), PB (True), DIR(True)
0091610514 RA (False), PB (False), DIR(True)
0091610642 RA (False), PB (False), DIR(True)
0091610664 RA (False), PB (True), DIR(True)
0091610716 RA (False), PB (True), DIR(True)
0091610747 RA (False), PB (False), DIR(True)
0091610915 RA (False), PB (True), DIR(True)
0101612076 RA (False), PB (False), DIR(True)
0101612157 RA (False), PB (False), DIR(True)
0101612368 RA (False), PB (False), DIR(True)
0101612394 RA (False), PB (False), DIR(True)
0101611009 RA (False), PB (True), DIR(True)
0101611542 RA (False), PB (False), DIR(True)
0091610973 RA (Fa

0141611073 RA (False), PB (False), DIR(True)
0141611101 RA (False), PB (False), DIR(True)
0131701867 RA (False), PB (False), DIR(True)
0131702021 RA (False), PB (True), DIR(True)
0131702170 RA (False), PB (False), DIR(True)
0131702223 RA (False), PB (False), DIR(True)
0131702264 RA (False), PB (False), DIR(True)
0141700830 RA (False), PB (False), DIR(True)
0141701128 RA (False), PB (False), DIR(True)
0141611251 RA (False), PB (False), DIR(True)
0141611596 RA (False), PB (False), DIR(True)
0131800188 RA (False), PB (False), DIR(True)
0131800407 RA (False), PB (False), DIR(True)
0141701494 RA (False), PB (False), DIR(True)
0141701634 RA (False), PB (False), DIR(True)
0141701642 RA (False), PB (False), DIR(True)
0141701680 RA (False), PB (False), DIR(True)
0141612043 RA (False), PB (False), DIR(True)
0131800802 RA (False), PB (False), DIR(True)
0141701759 RA (False), PB (True), DIR(True)
0141701801 RA (False), PB (False), DIR(True)
0141702268 RA (False), PB (False), DIR(True)
0141612397 R

0171801047 RA (False), PB (False), DIR(True)
0171801095 RA (False), PB (True), DIR(True)
0171801230 RA (False), PB (False), DIR(True)
0171801252 RA (False), PB (True), DIR(True)
0181613700 RA (False), PB (False), DIR(True)
0181613828 RA (False), PB (False), DIR(True)
0181700151 RA (False), PB (False), DIR(True)
0181700270 RA (False), PB (False), DIR(True)
0181700466 RA (False), PB (False), DIR(True)
0171702659 RA (False), PB (False), DIR(True)
0181700650 RA (False), PB (False), DIR(True)
0181700867 RA (False), PB (False), DIR(True)
0181700952 RA (False), PB (False), DIR(True)
0171801377 RA (False), PB (False), DIR(True)
0171801447 RA (False), PB (True), DIR(True)
0171801503 RA (False), PB (False), DIR(True)
0171801509 RA (False), PB (False), DIR(True)
0171801797 RA (False), PB (True), DIR(True)
0181701331 RA (False), PB (False), DIR(True)
0181701367 RA (False), PB (False), DIR(True)
0181610787 RA (False), PB (False), DIR(True)
0181610924 RA (False), PB (True), DIR(True)
0181611204 RA (

...

1051705532 RA (False), PB (False), DIR(True)
1051706087 RA (False), PB (False), DIR(True)
1051706129 RA (False), PB (False), DIR(True)
1051706190 RA (False), PB (False), DIR(True)
1051706453 RA (False), PB (False), DIR(True)
1051706499 RA (False), PB (False), DIR(True)
1051706532 RA (False), PB (False), DIR(True)
1051706622 RA (False), PB (False), DIR(True)
1051706626 RA (False), PB (False), DIR(True)
1051706884 RA (False), PB (False), DIR(True)
1051706992 RA (False), PB (False), DIR(True)
1051800388 RA (False), PB (False), DIR(True)
1051800460 RA (False), PB (False), DIR(True)
1051800512 RA (False), PB (False), DIR(True)
1051800640 RA (False), PB (False), DIR(True)
1051801066 RA (False), PB (False), DIR(True)
1051801256 RA (False), PB (False), DIR(True)
1051801478 RA (False), PB (False), DIR(True)
1051801697 RA (False), PB (False), DIR(True)
1051802066 RA (False), PB (False), DIR(True)
1051802067 RA (False), PB (False), DIR(True)
1051802441 RA (False), PB (False), DIR(True)
1051802510

1091611658 RA (False), PB (False), DIR(True)
1091611776 RA (False), PB (False), DIR(True)
1091611937 RA (False), PB (False), DIR(True)
1091611950 RA (False), PB (True), DIR(True)
1091612009 RA (False), PB (False), DIR(True)
1091612234 RA (False), PB (False), DIR(True)
1091612253 RA (False), PB (False), DIR(True)
1091612368 RA (False), PB (False), DIR(True)
1091612456 RA (False), PB (False), DIR(True)
1091612546 RA (False), PB (False), DIR(True)
1091612911 RA (False), PB (False), DIR(True)
1091612917 RA (False), PB (False), DIR(True)
1091613015 RA (False), PB (False), DIR(True)
1091613168 RA (False), PB (True), DIR(True)
1091613229 RA (False), PB (False), DIR(True)
1091613532 RA (False), PB (False), DIR(True)
1091613648 RA (False), PB (False), DIR(True)
1091613768 RA (False), PB (False), DIR(True)
1091613792 RA (False), PB (False), DIR(True)
1091613965 RA (False), PB (False), DIR(True)
1091614203 RA (False), PB (False), DIR(True)
1091614204 RA (False), PB (False), DIR(True)
1091614523 R

1111612426 RA (False), PB (False), DIR(True)
1111612686 RA (False), PB (False), DIR(True)
1111612751 RA (False), PB (False), DIR(True)
1111613035 RA (False), PB (False), DIR(True)
1111613146 RA (False), PB (False), DIR(True)
1111613202 RA (False), PB (False), DIR(True)
1111613336 RA (False), PB (False), DIR(True)
1111613425 RA (False), PB (False), DIR(True)
1111700041 RA (False), PB (False), DIR(True)
1111700174 RA (False), PB (False), DIR(True)
1111700331 RA (False), PB (False), DIR(True)
1111700342 RA (False), PB (False), DIR(True)
1111700568 RA (False), PB (False), DIR(True)
1111700661 RA (False), PB (False), DIR(True)
1111700994 RA (False), PB (False), DIR(True)
1111701124 RA (False), PB (False), DIR(True)
1111701164 RA (False), PB (False), DIR(True)
1111701461 RA (False), PB (False), DIR(True)
1111701505 RA (False), PB (False), DIR(True)
1111702686 RA (False), PB (False), DIR(True)
1111703156 RA (False), PB (False), DIR(True)
1111703206 RA (False), PB (False), DIR(True)
1111703208

1141613951 RA (False), PB (False), DIR(True)
1141614026 RA (False), PB (False), DIR(True)
1141614071 RA (False), PB (True), DIR(True)
1141700528 RA (False), PB (False), DIR(True)
1141700675 RA (False), PB (False), DIR(True)
1141700773 RA (False), PB (False), DIR(True)
1141700940 RA (False), PB (False), DIR(True)
1141701039 RA (False), PB (False), DIR(True)
1141701118 RA (False), PB (False), DIR(True)
1141701147 RA (False), PB (False), DIR(True)
1141701463 RA (False), PB (False), DIR(True)
1141701608 RA (False), PB (False), DIR(True)
1141701609 RA (False), PB (False), DIR(True)
1141701627 RA (False), PB (False), DIR(True)
1141701766 RA (False), PB (True), DIR(True)
1141701867 RA (False), PB (False), DIR(True)
1141701970 RA (False), PB (False), DIR(True)
1141702367 RA (False), PB (False), DIR(True)
1141702385 RA (False), PB (False), DIR(True)
1141702430 RA (False), PB (False), DIR(True)
1141702503 RA (False), PB (False), DIR(True)
1141702577 RA (False), PB (False), DIR(True)
1141702640 R

1211610849 RA (False), PB (False), DIR(True)
1211611038 RA (False), PB (False), DIR(True)
1211611201 RA (False), PB (False), DIR(True)
1211611487 RA (False), PB (False), DIR(True)
1211611510 RA (False), PB (True), DIR(True)
1211611985 RA (False), PB (False), DIR(True)
1211612066 RA (False), PB (False), DIR(True)
1211612162 RA (False), PB (False), DIR(True)
1211612610 RA (False), PB (False), DIR(True)
1211612690 RA (False), PB (False), DIR(True)
1211612864 RA (False), PB (False), DIR(True)
1211700246 RA (False), PB (False), DIR(True)
1211700355 RA (False), PB (False), DIR(True)
1211700551 RA (False), PB (False), DIR(True)
1211700568 RA (False), PB (False), DIR(True)
1211700809 RA (False), PB (False), DIR(True)
1211700857 RA (False), PB (False), DIR(True)
1211700910 RA (False), PB (False), DIR(True)
1211700988 RA (False), PB (False), DIR(True)
1211701006 RA (False), PB (False), DIR(True)
1211701179 RA (False), PB (False), DIR(True)
1211701185 RA (False), PB (False), DIR(True)
1211701233 

In [19]:
def approved_intersections(md):
    ints_w_warrant_met = dict()
    for master in md.keys():
        if md[master][-1].warrant():
            ints_w_warrant_met[master] = md[master][-1].get_12_month_crashes()
            # print master, len(md[master][-1].twelve_month_period())
    return ints_w_warrant_met

In [20]:
approved_masters = approved_intersections(masters)

1201611062 RA (False), PB (False), DIR(True)
1231611191 RA (False), PB (True), DIR(True)
1231610296 RA (False), PB (False), DIR(True)
1231800572 RA (False), PB (False), DIR(True)
1231700896 RA (False), PB (False), DIR(True)
1231800702 RA (False), PB (False), DIR(True)
1231610599 RA (False), PB (False), DIR(True)
1231800725 RA (False), PB (False), DIR(True)
1231610915 RA (False), PB (False), DIR(True)
1231801116 RA (False), PB (False), DIR(True)
1231800783 RA (False), PB (False), DIR(True)
1231700112 RA (False), PB (False), DIR(True)
1231700939 RA (False), PB (False), DIR(True)
1231801025 RA (False), PB (False), DIR(True)
1231701079 RA (False), PB (True), DIR(True)
1231800783 RA (False), PB (False), DIR(True)
1231700112 RA (False), PB (False), DIR(True)
1231700939 RA (False), PB (False), DIR(True)
1231801025 RA (False), PB (False), DIR(True)
1231701079 RA (False), PB (True), DIR(True)
1211612897 RA (False), PB (False), DIR(True)
1231701655 RA (False), PB (False), DIR(True)
1231800478 RA

0941702659 RA (False), PB (False), DIR(True)
1221701082 RA (False), PB (False), DIR(True)
1221611442 RA (False), PB (False), DIR(True)
1221701646 RA (False), PB (False), DIR(True)
1221612548 RA (False), PB (True), DIR(True)
1221800189 RA (False), PB (False), DIR(True)
1221701646 RA (False), PB (False), DIR(True)
1221612548 RA (False), PB (True), DIR(True)
1221800189 RA (False), PB (False), DIR(True)
1221611294 RA (False), PB (True), DIR(True)
1221611294 RA (False), PB (False), DIR(True)
1221703552 RA (False), PB (False), DIR(True)
1221612291 RA (False), PB (False), DIR(True)
1221703668 RA (False), PB (True), DIR(True)
1221701873 RA (False), PB (False), DIR(True)
1221611080 RA (False), PB (False), DIR(True)
1221704189 RA (False), PB (False), DIR(True)
1221800159 RA (False), PB (False), DIR(True)
1221612044 RA (False), PB (False), DIR(True)
1221800610 RA (False), PB (False), DIR(True)
1221701975 RA (False), PB (False), DIR(True)
1221702890 RA (False), PB (False), DIR(True)
1221703413 RA 

0661702725 RA (False), PB (False), DIR(True)
0661800853 RA (False), PB (False), DIR(True)
0661702721 RA (False), PB (False), DIR(True)
0661702103 RA (False), PB (False), DIR(True)
0621610691 RA (False), PB (False), DIR(True)
0621701380 RA (False), PB (False), DIR(True)
0621801598 RA (False), PB (True), DIR(True)
0621800658 RA (False), PB (False), DIR(True)
0621800659 RA (False), PB (False), DIR(True)
0621700893 RA (False), PB (False), DIR(True)
0621800749 RA (False), PB (False), DIR(True)
0621703184 RA (False), PB (False), DIR(True)
0621800144 RA (False), PB (False), DIR(True)
0621801574 RA (False), PB (False), DIR(True)
0621801628 RA (False), PB (False), DIR(True)
0621800012 RA (False), PB (False), DIR(True)
0621800012 RA (False), PB (False), DIR(True)
0621610807 RA (False), PB (False), DIR(True)
0621611023 RA (False), PB (False), DIR(True)
0621702533 RA (False), PB (False), DIR(True)
0661801259 RA (False), PB (False), DIR(True)
0661800823 RA (False), PB (False), DIR(True)
0661801535 

0841701568 RA (False), PB (False), DIR(True)
0841702654 RA (False), PB (False), DIR(True)
0841800138 RA (False), PB (False), DIR(True)
0841801375 RA (False), PB (False), DIR(True)
0841801725 RA (False), PB (False), DIR(True)
0841701190 RA (False), PB (False), DIR(True)
0841702775 RA (False), PB (False), DIR(True)
0841702504 RA (False), PB (True), DIR(True)
0841702497 RA (False), PB (True), DIR(True)
0841800694 RA (False), PB (False), DIR(True)
0841702504 RA (False), PB (True), DIR(True)
0841702497 RA (False), PB (True), DIR(True)
0841800694 RA (False), PB (False), DIR(True)
0841700924 RA (False), PB (False), DIR(True)
0841703212 RA (False), PB (False), DIR(True)
0841611420 RA (False), PB (False), DIR(True)
0841800692 RA (False), PB (False), DIR(True)
0841701046 RA (False), PB (True), DIR(True)
0071701245 RA (False), PB (False), DIR(True)
0071701089 RA (False), PB (False), DIR(True)
0071610724 RA (False), PB (False), DIR(True)
0071701408 RA (False), PB (True), DIR(True)
0071701215 RA (F

0701801508 RA (False), PB (False), DIR(True)
0701703083 RA (False), PB (False), DIR(True)
0701611440 RA (False), PB (True), DIR(True)
0701611440 RA (False), PB (True), DIR(True)
0701702982 RA (False), PB (False), DIR(True)
0701702982 RA (False), PB (False), DIR(True)
0701611625 RA (False), PB (False), DIR(True)
0701800089 RA (False), PB (False), DIR(True)
0711700975 RA (False), PB (True), DIR(True)
0701702015 RA (False), PB (False), DIR(True)
0711700483 RA (False), PB (False), DIR(True)
0701611625 RA (False), PB (False), DIR(True)
0701800089 RA (False), PB (False), DIR(True)
0711700975 RA (False), PB (True), DIR(True)
0701702015 RA (False), PB (False), DIR(True)
0711700483 RA (False), PB (False), DIR(True)
0701801491 RA (False), PB (False), DIR(True)
0701612485 RA (False), PB (False), DIR(True)
0701800050 RA (False), PB (False), DIR(True)
0701800050 RA (False), PB (False), DIR(True)
0701703089 RA (False), PB (False), DIR(True)
0701700546 RA (False), PB (True), DIR(True)
0701610904 RA (

...

1101612703 RA (False), PB (True), DIR(True)
1101612703 RA (False), PB (True), DIR(True)
1151703223 RA (False), PB (False), DIR(True)
1151703223 RA (False), PB (False), DIR(True)
1151802071 RA (False), PB (False), DIR(True)
1151702463 RA (False), PB (False), DIR(True)
1151612663 RA (False), PB (False), DIR(True)
1151801984 RA (False), PB (False), DIR(True)
1151702190 RA (False), PB (False), DIR(True)
1151702057 RA (False), PB (False), DIR(True)
1151702190 RA (False), PB (False), DIR(True)
1151702057 RA (False), PB (False), DIR(True)
1151611077 RA (False), PB (False), DIR(True)
1151702199 RA (False), PB (False), DIR(True)
1151701692 RA (False), PB (False), DIR(True)
1151611077 RA (False), PB (False), DIR(True)
1151702199 RA (False), PB (False), DIR(True)
1151701692 RA (False), PB (False), DIR(True)
1101701118 RA (False), PB (False), DIR(True)
1101610504 RA (False), PB (False), DIR(True)
1151700888 RA (False), PB (False), DIR(True)
1101800117 RA (False), PB (False), DIR(True)
1101700927 R

0471702761 RA (False), PB (False), DIR(True)
0471610534 RA (False), PB (False), DIR(True)
0471703089 RA (False), PB (False), DIR(True)
0471612456 RA (False), PB (False), DIR(True)
0471701181 RA (False), PB (False), DIR(True)
0471700404 RA (False), PB (False), DIR(True)
0471611142 RA (False), PB (False), DIR(True)
0471700535 RA (False), PB (False), DIR(True)
0471700146 RA (False), PB (False), DIR(True)
0471800161 RA (False), PB (False), DIR(True)
0471613318 RA (False), PB (False), DIR(True)
0471800636 RA (False), PB (False), DIR(True)
0471800636 RA (False), PB (False), DIR(True)
0471702181 RA (False), PB (False), DIR(True)
0471702879 RA (False), PB (False), DIR(True)
0471702181 RA (False), PB (False), DIR(True)
0471702879 RA (False), PB (False), DIR(True)
0471612493 RA (False), PB (False), DIR(True)
0471701300 RA (False), PB (False), DIR(True)
0471703847 RA (False), PB (False), DIR(True)
0471701300 RA (False), PB (False), DIR(True)
0471703847 RA (False), PB (False), DIR(True)
0471611024

0451702192 RA (False), PB (False), DIR(True)
0451702192 RA (False), PB (False), DIR(True)
0451701992 RA (False), PB (False), DIR(True)
0451700554 RA (False), PB (False), DIR(True)
0451704003 RA (False), PB (False), DIR(True)
0451700891 RA (False), PB (False), DIR(True)
0451610413 RA (False), PB (False), DIR(True)
0451800511 RA (False), PB (False), DIR(True)
0451610709 RA (False), PB (False), DIR(True)
0451801519 RA (False), PB (False), DIR(True)
0451800487 RA (False), PB (False), DIR(True)
0451800146 RA (False), PB (False), DIR(True)
0451701106 RA (False), PB (False), DIR(True)
0451702253 RA (False), PB (False), DIR(True)
0451702298 RA (False), PB (True), DIR(True)
0451612306 RA (False), PB (False), DIR(True)
0451703248 RA (False), PB (False), DIR(True)
0451801057 RA (False), PB (False), DIR(True)
0451612306 RA (False), PB (False), DIR(True)
0451703248 RA (False), PB (False), DIR(True)
0451801057 RA (False), PB (False), DIR(True)
0451801326 RA (False), PB (False), DIR(True)
0451700575 

0791700588 RA (False), PB (True), DIR(True)
0901612819 RA (False), PB (False), DIR(True)
1141700832 RA (False), PB (False), DIR(True)
1141702306 RA (False), PB (False), DIR(True)
1141800297 RA (False), PB (False), DIR(True)
1141700832 RA (False), PB (False), DIR(True)
1141702306 RA (False), PB (False), DIR(True)
1141800297 RA (False), PB (False), DIR(True)
1041612524 RA (False), PB (False), DIR(True)
1041702604 RA (False), PB (False), DIR(True)
0901702886 RA (False), PB (False), DIR(True)
0901702886 RA (False), PB (False), DIR(True)
0881611040 RA (False), PB (False), DIR(True)
0631700344 RA (False), PB (False), DIR(True)
0681611944 RA (False), PB (False), DIR(True)
0681700459 RA (False), PB (False), DIR(True)
0661701018 RA (False), PB (False), DIR(True)
0781800208 RA (False), PB (False), DIR(True)
0781610932 RA (False), PB (True), DIR(True)
0941801017 RA (False), PB (False), DIR(True)
0941800694 RA (False), PB (False), DIR(True)
0941701039 RA (False), PB (False), DIR(True)
0941610845 R

1081703654 RA (False), PB (False), DIR(True)
1081611560 RA (False), PB (True), DIR(True)
1081702859 RA (False), PB (False), DIR(True)
1081612024 RA (False), PB (False), DIR(True)
1081704657 RA (False), PB (True), DIR(True)
0631800667 RA (False), PB (False), DIR(True)
0671612600 RA (False), PB (False), DIR(True)
0671700950 RA (False), PB (False), DIR(True)
0671613200 RA (False), PB (False), DIR(True)
0671612600 RA (False), PB (False), DIR(True)
0671700950 RA (False), PB (False), DIR(True)
0671613200 RA (False), PB (False), DIR(True)
0251611557 RA (False), PB (True), DIR(True)
0251801058 RA (False), PB (True), DIR(True)
0251611557 RA (False), PB (True), DIR(True)
0251801058 RA (False), PB (True), DIR(True)
1121611317 RA (False), PB (False), DIR(True)
1121800762 RA (False), PB (False), DIR(True)
0411611201 RA (False), PB (False), DIR(True)
0401703196 RA (False), PB (False), DIR(True)
1031701410 RA (False), PB (False), DIR(True)
1031701410 RA (False), PB (False), DIR(True)
0501701387 RA (F

In [24]:
def get_node_details(pg):
    data = dict()
    cur = pg.conn.cursor()
    cur.execute("""select  nodeid, masterid, st1, st2, st_x(geom) as x, st_y(geom) as y
                    from node n
                    join (
                        select nodeidfrom, min(st1) as st1, max(st2) as st2
                        from(
                            select s1.nodeidfrom, s1.street as st1, s2.street  as st2
                            from lion as s1 join lion as s2
                            on s1.nodeidfrom = s2.nodeidto
                            where s1.street != s2.street
                            and s1.featuretyp not in ('1', '2', '3', '7')
                            and s2.featuretyp not in ('1', '2', '3', '7')
                        ) as names_ group by nodeidfrom
                    ) as st_names on n.nodeid::int = st_names.nodeidfrom::int
                """)
    for row in cur.fetchall():
        nodeid, masterid, st1, st2, x, y = row
        data[nodeid] = [st1, st2, x, y]
    del cur
    return data

In [25]:
node_data = get_node_details(pgo)



In [26]:
for i in node_data:
    if i == 33755:
     print node_data[i]

['ROCKAWAY BOULEVARD', '110 STREET', 1031726.09807865, 186224.401813105]


In [27]:
def intersection_summaries(approved, mids, nds):
    summary = list()
    for m in approved.keys():
        crashes = len([c for c in approved[m]])
        ped_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 1])
        bike_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 2])
        mvo_inj = sum([approved[m][c].inj for c in approved[m] if approved[m][c].mode == 3])
        right_angles = len([approved[m][c].crashid for c in approved[m] if approved[m][c].col_typ in ('4', '04')])
        date_ranges = [min([approved[m][c].ac_date for c in approved[m]]),
                       max([approved[m][c].ac_date for c in approved[m]])]
        print 'Master %i (%s - %s):\n\tNodes %s \n\t%i crashes\n\t%i ped injuries\n\t%i bike' \
              'injuries\n\t%i mvo injuries\n\t%i right angle crashes' % (m, date_ranges[0].strftime('%m/%d/%Y'),
                                                                         date_ranges[1].strftime('%m/%d/%Y'),
                                                                         str(mids[m][-1].nodeid_list),
                                                                         crashes, ped_inj, bike_inj, mvo_inj,
                                                                         right_angles)
        if len(mids[m]) > 2:
            for node in mids[m][:-1]:
                if node in nds.keys():
                    s1, s2, x, y = nds[node]
                else:
                    s1, s2, x, y = '', '', 0, 0
                summary.append([m, node, crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                                right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
        else:
            if mids[m][0] in nds.keys():
                s1, s2, x, y = nds[mids[m][0]]
            else:
                s1, s2, x, y = '', '', 0, 0
            summary.append([m, mids[m][0], crashes, ped_inj, bike_inj, mvo_inj, ped_inj+bike_inj+mvo_inj,
                            right_angles, date_ranges[0], date_ranges[1], s1, s2, x, y])
    return ['MasterID', 'NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs',
            'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y'], summary

In [28]:
headers, crash_data = intersection_summaries(approved_masters, masters, node_data)


Master 90112 (06/26/2017 - 03/17/2018):
	Nodes [50918, 50920, 81085, 83556, 90112, 90115] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	2 mvo injuries
	4 right angle crashes
Master 32786 (08/18/2017 - 05/16/2018):
	Nodes [32786] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	2 mvo injuries
	5 right angle crashes
Master 32794 (07/28/2017 - 06/15/2018):
	Nodes [32794, 80889, 81329] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	6 mvo injuries
	3 right angle crashes
Master 106526 (06/29/2017 - 06/25/2018):
	Nodes [9046427, 32668, 76699, 83058, 83334, 106518, 106526, 106527] 
	13 crashes
	0 ped injuries
	0 bikeinjuries
	4 mvo injuries
	7 right angle crashes
Master 49183 (08/24/2017 - 07/16/2018):
	Nodes [49183] 
	5 crashes
	4 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 32802 (09/04/2017 - 05/26/2018):
	Nodes [32802] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	2 mvo injuries
	6 right angle crashes
Master 65592 (04/19/2017 - 11/30/2017):
	Nodes [26409, 65592] 
	7 c

Master 41276 (06/16/2017 - 04/26/2018):
	Nodes [41276] 
	14 crashes
	0 ped injuries
	0 bikeinjuries
	7 mvo injuries
	14 right angle crashes
Master 65860 (09/02/2017 - 07/13/2018):
	Nodes [26973, 26974, 26977, 65860] 
	7 crashes
	3 ped injuries
	1 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 16725 (10/01/2017 - 07/24/2018):
	Nodes [16725] 
	7 crashes
	2 ped injuries
	2 bikeinjuries
	1 mvo injuries
	1 right angle crashes
Master 90455 (08/18/2017 - 07/16/2018):
	Nodes [26263, 89751, 90455] 
	10 crashes
	2 ped injuries
	0 bikeinjuries
	11 mvo injuries
	3 right angle crashes
Master 16731 (06/06/2017 - 05/19/2018):
	Nodes [16731] 
	5 crashes
	2 ped injuries
	1 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 41322 (07/12/2017 - 06/23/2018):
	Nodes [41322] 
	11 crashes
	7 ped injuries
	1 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 15080 (05/12/2016 - 10/08/2016):
	Nodes [15080, 80135, 80727] 
	6 crashes
	1 ped injuries
	3 bikeinjuries
	0 mvo injuries
	0 

Master 16942 (11/27/2017 - 06/30/2018):
	Nodes [16942] 
	5 crashes
	1 ped injuries
	1 bikeinjuries
	3 mvo injuries
	1 right angle crashes
Master 41525 (01/30/2017 - 01/24/2018):
	Nodes [41525] 
	9 crashes
	1 ped injuries
	0 bikeinjuries
	3 mvo injuries
	1 right angle crashes
Master 41534 (09/13/2016 - 04/24/2017):
	Nodes [41532, 41533, 41534] 
	5 crashes
	2 ped injuries
	1 bikeinjuries
	1 mvo injuries
	0 right angle crashes
Master 57920 (11/24/2017 - 07/19/2018):
	Nodes [57920, 84673, 91910] 
	7 crashes
	1 ped injuries
	0 bikeinjuries
	1 mvo injuries
	2 right angle crashes
Master 41539 (10/08/2017 - 06/29/2018):
	Nodes [41539] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	3 right angle crashes
Master 16969 (07/06/2017 - 07/02/2018):
	Nodes [16969] 
	5 crashes
	2 ped injuries
	1 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 41549 (05/09/2017 - 03/18/2018):
	Nodes [41549] 
	5 crashes
	3 ped injuries
	1 bikeinjuries
	1 mvo injuries
	2 right angle crashes
Master

Master 9038640 (05/15/2017 - 02/23/2018):
	Nodes [2639, 9038640] 
	8 crashes
	0 ped injuries
	0 bikeinjuries
	0 mvo injuries
	5 right angle crashes
Master 8999 (09/02/2017 - 07/24/2018):
	Nodes [8999] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	1 mvo injuries
	3 right angle crashes
Master 90921 (11/05/2016 - 10/03/2017):
	Nodes [61326, 81295, 83617, 90917, 90921, 90922] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	1 right angle crashes
Master 104465 (08/08/2017 - 07/05/2018):
	Nodes [18680, 18682, 18687, 18688, 65567, 104436, 104465] 
	5 crashes
	1 ped injuries
	1 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 41777 (07/07/2017 - 07/02/2018):
	Nodes [41776, 41777] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	1 mvo injuries
	1 right angle crashes
Master 17209 (03/18/2017 - 02/11/2018):
	Nodes [17209] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	10 mvo injuries
	4 right angle crashes
Master 41786 (07/15/2017 - 05/16/2018):
	Nodes [41786] 
	5 crashes
	1 ped in

Master 17396 (09/01/2017 - 07/23/2018):
	Nodes [17396, 87553, 87850] 
	8 crashes
	1 ped injuries
	2 bikeinjuries
	1 mvo injuries
	5 right angle crashes
Master 17399 (07/20/2017 - 06/11/2018):
	Nodes [17399, 87549, 87847] 
	10 crashes
	2 ped injuries
	0 bikeinjuries
	3 mvo injuries
	6 right angle crashes
Master 17400 (07/28/2017 - 06/03/2018):
	Nodes [17400] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	0 mvo injuries
	3 right angle crashes
Master 17402 (09/15/2017 - 07/23/2018):
	Nodes [17402, 87545, 87842] 
	6 crashes
	2 ped injuries
	1 bikeinjuries
	5 mvo injuries
	3 right angle crashes
Master 25603 (08/25/2017 - 07/24/2018):
	Nodes [25603] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 9044996 (03/04/2017 - 02/06/2018):
	Nodes [9044996, 26387] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	1 mvo injuries
	2 right angle crashes
Master 25608 (11/27/2017 - 07/17/2018):
	Nodes [25608] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	11 mvo injuries
	5

...

Master 9034824 (08/05/2017 - 07/18/2018):
	Nodes [35478, 9034824] 
	6 crashes
	3 ped injuries
	0 bikeinjuries
	1 mvo injuries
	1 right angle crashes
Master 31823 (11/20/2017 - 07/29/2018):
	Nodes [31823, 82621, 83059] 
	6 crashes
	0 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 40024 (04/20/2017 - 03/13/2018):
	Nodes [40024] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	5 mvo injuries
	6 right angle crashes
Master 23644 (06/01/2017 - 05/24/2018):
	Nodes [23644] 
	5 crashes
	5 ped injuries
	1 bikeinjuries
	2 mvo injuries
	3 right angle crashes
Master 40030 (08/01/2017 - 07/09/2018):
	Nodes [40029, 40030] 
	5 crashes
	2 ped injuries
	0 bikeinjuries
	0 mvo injuries
	3 right angle crashes
Master 92689 (07/12/2017 - 06/14/2018):
	Nodes [16758, 16759, 92689] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	2 mvo injuries
	2 right angle crashes
Master 48232 (03/02/2017 - 01/29/2018):
	Nodes [48232] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	8 mvo injuries
	1 right an

Master 40248 (03/25/2017 - 01/07/2018):
	Nodes [40248, 85839, 86357] 
	5 crashes
	0 ped injuries
	0 bikeinjuries
	4 mvo injuries
	3 right angle crashes
Master 48446 (10/02/2017 - 07/11/2018):
	Nodes [48446] 
	6 crashes
	1 ped injuries
	0 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 48450 (12/10/2016 - 10/24/2017):
	Nodes [48450] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 7504 (09/04/2017 - 07/25/2018):
	Nodes [7504, 83761, 87454] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	2 mvo injuries
	5 right angle crashes
Master 15704 (07/20/2017 - 06/18/2018):
	Nodes [15704] 
	5 crashes
	4 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 23897 (03/06/2017 - 02/27/2018):
	Nodes [23897, 78200, 78202] 
	7 crashes
	0 ped injuries
	2 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 40282 (05/18/2017 - 03/03/2018):
	Nodes [40282, 92408, 92411] 
	8 crashes
	0 ped injuries
	1 bikeinjuries
	2 mvo injuries
	0 

Master 48663 (02/01/2017 - 12/03/2017):
	Nodes [48663] 
	6 crashes
	4 ped injuries
	0 bikeinjuries
	1 mvo injuries
	2 right angle crashes
Master 48665 (06/19/2017 - 06/10/2018):
	Nodes [48665] 
	5 crashes
	5 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 48666 (04/10/2017 - 03/18/2018):
	Nodes [48666] 
	14 crashes
	5 ped injuries
	0 bikeinjuries
	1 mvo injuries
	0 right angle crashes
Master 24091 (09/17/2017 - 07/19/2018):
	Nodes [24090, 24091] 
	7 crashes
	3 ped injuries
	0 bikeinjuries
	4 mvo injuries
	2 right angle crashes
Master 24093 (08/05/2017 - 08/01/2018):
	Nodes [24093] 
	11 crashes
	3 ped injuries
	2 bikeinjuries
	1 mvo injuries
	1 right angle crashes
Master 48677 (02/22/2017 - 01/25/2018):
	Nodes [48677] 
	6 crashes
	3 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 81446 (06/20/2017 - 04/29/2018):
	Nodes [27928, 81446] 
	10 crashes
	1 ped injuries
	1 bikeinjuries
	0 mvo injuries
	1 right angle crashes
Master 48681 (02/

Master 24276 (10/17/2017 - 08/05/2018):
	Nodes [24276, 78425, 78502] 
	10 crashes
	9 ped injuries
	0 bikeinjuries
	3 mvo injuries
	3 right angle crashes
Master 24280 (10/31/2017 - 07/22/2018):
	Nodes [24280, 78346, 78504] 
	5 crashes
	3 ped injuries
	1 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 29988 (09/17/2017 - 05/14/2018):
	Nodes [29988] 
	8 crashes
	4 ped injuries
	0 bikeinjuries
	1 mvo injuries
	0 right angle crashes
Master 24282 (07/30/2017 - 07/11/2018):
	Nodes [24282, 78344, 78906] 
	7 crashes
	3 ped injuries
	2 bikeinjuries
	0 mvo injuries
	2 right angle crashes
Master 48864 (10/24/2017 - 06/18/2018):
	Nodes [48862, 48864] 
	5 crashes
	3 ped injuries
	1 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 24290 (09/14/2017 - 06/29/2018):
	Nodes [24290, 77884, 78067] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	7 mvo injuries
	1 right angle crashes
Master 24300 (09/14/2017 - 08/03/2018):
	Nodes [24300] 
	6 crashes
	3 ped injuries
	3 bikeinjuries
	0 mvo in

Master 31533 (07/25/2017 - 03/22/2018):
	Nodes [31533] 
	5 crashes
	0 ped injuries
	1 bikeinjuries
	4 mvo injuries
	4 right angle crashes
Master 40931 (05/03/2017 - 04/12/2018):
	Nodes [40931] 
	7 crashes
	0 ped injuries
	0 bikeinjuries
	3 mvo injuries
	6 right angle crashes
Master 9043949 (11/10/2016 - 07/07/2017):
	Nodes [7077, 9043949] 
	5 crashes
	2 ped injuries
	0 bikeinjuries
	2 mvo injuries
	1 right angle crashes
Master 49145 (09/19/2016 - 12/18/2016):
	Nodes [49145] 
	5 crashes
	1 ped injuries
	0 bikeinjuries
	0 mvo injuries
	0 right angle crashes
Master 9040558 (09/13/2017 - 07/16/2018):
	Nodes [42964, 79443, 101104, 105044, 9040556, 9040557, 9040558] 
	6 crashes
	2 ped injuries
	0 bikeinjuries
	0 mvo injuries
	1 right angle crashes


In [32]:
for i in range(5):
    print headers[]


SyntaxError: invalid syntax (<ipython-input-32-e67a03377f2b>, line 2)

In [48]:
print headers[1:14]

['NodeID', 'Crashes', 'Ped_Injs', 'Bike_Injs', 'MVO_Injs', 'Total_Injs', 'Right_Angle_Crashes', 'Earliest_Date', 'Latest_Date', 'Street1', 'Street2', 'X', 'Y']


In [56]:
print approved_masters.keys()

[26626, 18439, 18440, 32786, 20500, 36885, 36888, 55321, 59422, 36896, 26657, 32802, 45097, 28718, 34864, 28723, 36918, 48137, 34872, 18494, 34880, 16449, 55363, 26708, 34903, 57443, 16488, 10354, 48147, 20596, 55413, 8313, 53375, 34951, 34954, 29720, 104473, 27332, 18587, 69788, 65705, 37034, 53423, 39092, 53430, 28862, 40310, 35014, 55495, 6178, 53464, 26851, 28901, 28904, 39148, 28909, 28910, 57588, 35066, 24827, 51454, 24839, 65802, 67318, 10516, 49436, 20768, 53537, 37160, 59689, 54430, 45365, 37174, 41276, 14656, 34870, 39238, 53575, 39241, 35150, 26959, 39249, 11662, 39254, 39264, 6500, 53605, 26982, 55663, 41333, 53626, 46181, 29056, 35201, 47490, 14724, 27632, 39314, 39321, 14744, 16793, 14752, 78919, 55726, 47859, 25013, 41399, 53688, 42399, 41404, 41408, 27077, 27078, 41478, 47564, 55759, 20944, 41430, 35303, 35306, 31211, 39406, 31215, 27123, 41464, 23035, 37372, 41470, 31232, 2561, 41475, 29190, 27143, 39432, 40024, 53783, 12830, 12832, 12834, 31270, 61667, 12841, 29228, 2

In [31]:
print approved_masters[33755]


{u'1061800934': <__main__.Crash instance at 0x12045C60>, u'1061800876': <__main__.Crash instance at 0x12E00A08>, u'1061801226': <__main__.Crash instance at 0x12E243A0>, u'1061802133': <__main__.Crash instance at 0x12E2A2B0>, u'1061801174': <__main__.Crash instance at 0x12E00D50>, u'1061702456': <__main__.Crash instance at 0x11F9C1C0>, u'1061801172': <__main__.Crash instance at 0x156CC2B0>, u'1061801646': <__main__.Crash instance at 0x156CC990>, u'1061801178': <__main__.Crash instance at 0x1204A148>, u'1061800610': <__main__.Crash instance at 0x12E209E0>, u'1061703407': <__main__.Crash instance at 0x12E1A1C0>, u'1061703273': <__main__.Crash instance at 0x156BCCB0>, u'1061703297': <__main__.Crash instance at 0x12DA9E18>, u'1061703564': <__main__.Crash instance at 0x156C2260>, u'1061801623': <__main__.Crash instance at 0x12E24990>, u'1061801813': <__main__.Crash instance at 0x156CCC88>, u'1061703866': <__main__.Crash instance at 0x156C2788>, u'1061800021': <__main__.Crash instance at 0x12

In [33]:
for i in approved_masters[32668]:
    
        print(i)

KeyError: 32668